# Install the required libraries


In [ ]:
!pip install numpy pandas matplotlib seaborn scikit-learn ipywidgets


# Setup and Imports:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# For interactive widgets
!pip install -q ipywidgets
import ipywidgets as widgets
from IPython.display import display

# Load and Explore Data:

In [ ]:
url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
df = pd.read_csv(url)


print("=== Dataset Overview ===")
print("Shape:", df.shape)
print("\nFirst 5 rows:\n", df.head())
print("\nData Summary:\n")
print(df.info())
print("\nStatistical Summary:\n", df.describe())
print("\nMissing values:", df.isnull().sum().sum())

# Visualize Correlation:

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Features')
plt.show()

# Data Preprocessing:

In [ ]:
X = df.drop('medv', axis=1)
y = df['medv'] * 1000  # Convert target to dollars

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Multiple Models:

In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Support Vector Regression': SVR()
}

results = {}

print("\n=== Model Training and Evaluation ===")
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    results[name] = {
        'MSE': mean_squared_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAE': mean_absolute_error(y_test, y_pred),
        'R² Score': r2_score(y_test, y_pred)
    }

# Show model results
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values(by='R² Score', ascending=False)

print("\nModel Performance Comparison:")
display(results_df)


# Hyperparameter Tuning (Random Forest):

In [ ]:
print("\n=== Hyperparameter Tuning (Random Forest) ===")
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(
    RandomForestRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1
)
grid_search.fit(X_train_scaled, y_train)

best_rf = grid_search.best_estimator_

print("Best Parameters:", grid_search.best_params_)


# Feature Importance:

In [ ]:
feature_importance = best_rf.feature_importances_
features = X.columns

importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(12, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Feature Importance (Random Forest)')
plt.show()


# Interactive Prediction Tool:

In [ ]:
print("\n=== Interactive Housing Price Predictor ===")

# Create input widgets
input_widgets = {
    'rm': widgets.FloatSlider(value=6, min=3, max=9, step=0.5, description='Avg rooms:'),
    'lstat': widgets.FloatSlider(value=12, min=2, max=38, step=1, description='Low status %:'),
    'ptratio': widgets.FloatSlider(value=19, min=12, max=22, step=0.5, description='Pupil-teacher ratio:'),
    'dis': widgets.FloatSlider(value=3.5, min=1, max=12, step=0.5, description='Distance to Employment:'),
    'nox': widgets.FloatSlider(value=0.5, min=0.3, max=0.9, step=0.05, description='Nitric oxide:'),
    'crim': widgets.FloatSlider(value=0.5, min=0, max=90, step=1, description='Crime rate:'),
    'chas': widgets.Dropdown(options=[0, 1], value=0, description='Near Charles River:')
}

# Default values for remaining features
default_values = {
    'zn': 0,
    'indus': 11,
    'age': 68,
    'rad': 5,
    'tax': 300,
    'b': 396
}

# Create form layout
form_items = [widgets.HTML(value="<h3>Enter Property Details:</h3>")]
form_items.extend(input_widgets.values())
form = widgets.VBox(form_items)

predict_btn = widgets.Button(description="Predict Price", button_style='success')
output = widgets.Output()

# Prediction function
def make_prediction(b):
    input_values = {**{k: w.value for k, w in input_widgets.items()}, **default_values}
    input_df = pd.DataFrame([input_values])[X.columns]
    scaled_input = scaler.transform(input_df)
    prediction = best_rf.predict(scaled_input)[0]

    output.clear_output()
    with output:
        print(f"\n🏠 Predicted Home Value: ${prediction:,.2f}")
        print("\n🔹 Key Factors Considered:")
        for key, value in input_values.items():
            print(f"- {key}: {value}")

predict_btn.on_click(make_prediction)

# Display form
display(widgets.VBox([form, predict_btn, output]))
